In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from __future__ import division

In [5]:
def getSessionRequest(email, password):
    session_requests = requests.session()
    login_url = 'https://www.alexa.com/secure/login'
    credentials = {'email': email, 'password': password}
    logged_in = session_requests.post(login_url, data=credentials)
    return session_requests

In [6]:
def getTableFromDomain(domain, session_requests):
    BASE_URL = 'http://www.alexa.com/siteinfo/'
    url = BASE_URL + domain
    response = session_requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    table = soup.body.find('div', attrs={'id': 'demographics-content'})
    return table

In [7]:
session_request = getSessionRequest('alex@kazim.org', 'y0ur0ck!')
table = getTableFromDomain('cnn.com', session_request)

In [24]:
def get_real_demographic(table):
    classs=table.find_all('span',{'class':'pybar-bg'})
    spanstring = []
    for x in classs:
        spanstring.append(str(x))
    
    words = []
    for item in spanstring:
        word=item.split()
        words.append(word)
    
    percentage = []
    for word in words:
        for i in word:
            if "width" in i:
                percentage.append(i)
            
    final_percentage = []
    for percent in percentage:
        final_percentage.append(percent[(percent.index(":")+1):percent.index("%")])
    final_percentage = map(int, final_percentage)
    
    real_demo=[0]*28
    i = 0
    count = 0
    while i < 28:
        real_demo[i] = (final_percentage[count]+final_percentage[count+1])/200
        i = i + 1
        count = count + 2
    return real_demo

In [25]:
real_demo=get_real_demographic(table)
len(real_demo)

28

In [26]:
def makeDfFromRealDemo(domain_name, real_demo):
    dict = {domain_name:real_demo}
    df=pd.DataFrame.from_dict(dict, orient='index', dtype=None)
    return df

In [27]:
print makeDfFromRealDemo("cnn.com",real_demo)

           0     1      2     3     4     5     6     7      8      9   ...   \
cnn.com  0.26  0.96  0.435  0.41  0.57  0.67  0.53  0.58  0.685  0.365  ...    

            18    19    20    21    22     23     24     25    26    27  
cnn.com  0.315  0.99  0.39  0.06  0.07  0.725  0.965  0.055  0.03  0.05  

[1 rows x 28 columns]


In [28]:
customer1 = pd.read_csv('domain_count.txt', sep='\t',header = None, names=["domain","count"])
final_df = pd.DataFrame()

In [36]:
count1=0
for single_domain in customer1["domain"][0:100]:
    table = getTableFromDomain(single_domain, session_request)
    real_demo = get_real_demographic(table)
    df = makeDfFromRealDemo(single_domain, real_demo)
    final_df=final_df.append(df)
    count1+=1
    print count1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


IndexError: list index out of range

In [32]:
final_df.columns = ["male", "female", "18-24", "25-34", "35-44", "45-54", "55-64", "65+", "Has Children", "No Children", 
"No_college", "Some_college", "Graduate_School", "College", "$0 - $30K", "$30K - $60K", "$60K - $100K", "$100K+", "Home", "School",
"Work", "African", "African American", "Asian", "Caucasian", "Hispanic", "Middle Eastern", "Other"]
final_df["Count"] = customer1["count"][0:15].values

In [33]:
pd.set_option('display.max_columns', 29)
final_df

,male,female,18-24,25-34,35-44,45-54,55-64,65+,Has Children,No Children,No_college,Some_college,Graduate_School,College,$0 - $30K,$30K - $60K,$60K - $100K,$100K+,Home,School,Work,African,African American,Asian,Caucasian,Hispanic,Middle Eastern,Other,Count
coolmath-games.com,0.260,0.960,0.435,0.410,0.570,0.670,0.530,0.580,0.685,0.365,0.810,0.830,0.250,0.420,0.145,0.395,0.905,0.815,0.315,0.990,0.390,0.060,0.070,0.725,0.965,0.055,0.030,0.050,9268406
scribol.com,0.560,0.445,0.230,0.345,0.485,0.805,0.990,0.990,0.470,0.530,0.190,0.525,0.480,0.715,0.285,0.570,0.465,0.815,0.540,0.045,0.710,0.145,0.820,0.105,0.990,0.490,0.300,0.340,7639125
offbeat.topix.com,0.310,0.810,0.390,0.450,0.535,0.640,0.625,0.660,0.530,0.470,0.600,0.725,0.315,0.490,0.205,0.475,0.760,0.730,0.385,0.945,0.445,0.145,0.240,0.480,0.990,0.170,0.075,0.120,6034821
roblox.com,0.330,0.760,0.490,0.410,0.550,0.620,0.525,0.515,0.675,0.370,0.745,0.760,0.295,0.440,0.225,0.445,0.790,0.725,0.400,0.990,0.370,0.195,0.195,0.620,0.835,0.200,0.155,0.180,5815526
answers.com,0.365,0.685,0.445,0.410,0.515,0.620,0.725,0.760,0.490,0.490,0.515,0.660,0.390,0.490,0.295,0.490,0.660,0.655,0.445,0.740,0.455,0.215,0.295,0.475,0.990,0.255,0.125,0.200,5243867
video.foxnews.com,0.510,0.490,0.340,0.425,0.510,0.680,0.855,0.855,0.470,0.535,0.380,0.580,0.415,0.590,0.220,0.510,0.640,0.800,0.445,0.490,0.570,0.240,0.400,0.270,0.990,0.405,0.150,0.235,3822694
pawsome.topix.com,0.310,0.810,0.390,0.450,0.535,0.640,0.625,0.660,0.530,0.470,0.600,0.725,0.315,0.490,0.205,0.475,0.760,0.730,0.385,0.945,0.445,0.145,0.240,0.480,0.990,0.170,0.075,0.120,3792292
games.aarp.org,0.270,0.920,0.350,0.390,0.510,0.705,0.890,0.990,0.575,0.435,0.705,0.800,0.290,0.430,0.150,0.420,0.840,0.830,0.345,0.990,0.415,0.065,0.155,0.595,0.990,0.150,0.030,0.055,3722262
y8.com,0.680,0.365,0.605,0.430,0.585,0.405,0.360,0.520,0.790,0.315,0.390,0.515,0.510,0.545,0.815,0.490,0.320,0.310,0.810,0.310,0.345,0.990,0.600,0.470,0.240,0.705,0.515,0.895,3704292
youtube.com,0.665,0.375,0.475,0.535,0.490,0.465,0.510,0.455,0.445,0.560,0.285,0.390,0.650,0.555,0.645,0.575,0.350,0.395,0.585,0.180,0.550,0.755,0.415,0.360,0.465,0.770,0.545,0.775,3613149
